In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
import geopandas as gpd

Reading in the file needed to create a map of the United States. The file comes from the Natural Earth dataset, which is a public domain map dataset that provides various geographical information. 

In [2]:
gdf = gpd.read_file(r"C:\Users\andre\OneDrive\Documents\Personal\ne_110m_admin_1_states_provinces.zip")
gdf.head()

,featurecla,scalerank,adm1_code,diss_me,iso_3166_2,wikipedia,iso_a2,adm0_sr,name,name_alt,...,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC,geometry
0,Admin-1 scale rank,2,USA-3514,3514,US-MN,http://en.wikipedia.org/wiki/Minnesota,US,1,Minnesota,MN|Minn.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-89.95766 47.28691, -90.13175 47.292..."
1,Admin-1 scale rank,2,USA-3515,3515,US-MT,http://en.wikipedia.org/wiki/Montana,US,1,Montana,MT|Mont.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-116.04823 49.00037, -113.0595 49.00..."
2,Admin-1 scale rank,2,USA-3516,3516,US-ND,http://en.wikipedia.org/wiki/North_Dakota,US,1,North Dakota,ND|N.D.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-97.22894 49.00089, -97.21414 48.902..."
3,Admin-1 scale rank,2,USA-3517,3517,US-HI,http://en.wikipedia.org/wiki/Hawaii,US,8,Hawaii,HI|Hawaii,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-155.93665 19.05939, -155.9080..."
4,Admin-1 scale rank,2,USA-3518,3518,US-ID,http://en.wikipedia.org/wiki/Idaho,US,1,Idaho,ID|Idaho,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-116.04823 49.00037, -115.9678 47.95..."


Reading in a dataset that contains information about the US Elections from 1976 to 2020. This dataset was obtained from Kaggle.com.

In [3]:
df = pd.read_csv('PresidentialData.csv')
df

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,504070,1182850,20210113,NaN,REPUBLICAN
2,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,False,9198,1182850,20210113,NaN,OTHER
3,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,False,6669,1182850,20210113,NaN,OTHER
4,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"HALL, GUS",COMMUNIST PARTY USE,False,1954,1182850,20210113,NaN,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4282,2020,WYOMING,WY,56,83,68,US PRESIDENT,"JORGENSEN, JO",LIBERTARIAN,False,5768,278503,20210113,NaN,LIBERTARIAN
4283,2020,WYOMING,WY,56,83,68,US PRESIDENT,"PIERCE, BROCK",INDEPENDENT,False,2208,278503,20210113,NaN,OTHER
4284,2020,WYOMING,WY,56,83,68,US PRESIDENT,NaN,NaN,True,1739,278503,20210113,NaN,OTHER
4285,2020,WYOMING,WY,56,83,68,US PRESIDENT,OVERVOTES,NaN,False,279,278503,20210113,NaN,OTHER


Basic Data Cleaning

In [4]:
remove = 'DISTRICT OF COLUMBIA'
df = df[~df['state'].str.contains(remove)]
df = df.drop(columns=['writein', 'office', 'version', 'notes'])
df

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,candidatevotes,totalvotes,party_simplified
0,1976,ALABAMA,AL,1,63,41,"CARTER, JIMMY",DEMOCRAT,659170,1182850,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,"FORD, GERALD",REPUBLICAN,504070,1182850,REPUBLICAN
2,1976,ALABAMA,AL,1,63,41,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,9198,1182850,OTHER
3,1976,ALABAMA,AL,1,63,41,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,6669,1182850,OTHER
4,1976,ALABAMA,AL,1,63,41,"HALL, GUS",COMMUNIST PARTY USE,1954,1182850,OTHER
...,...,...,...,...,...,...,...,...,...,...,...
4282,2020,WYOMING,WY,56,83,68,"JORGENSEN, JO",LIBERTARIAN,5768,278503,LIBERTARIAN
4283,2020,WYOMING,WY,56,83,68,"PIERCE, BROCK",INDEPENDENT,2208,278503,OTHER
4284,2020,WYOMING,WY,56,83,68,NaN,NaN,1739,278503,OTHER
4285,2020,WYOMING,WY,56,83,68,OVERVOTES,NaN,279,278503,OTHER


In [5]:
# To lowercase all states and then capitalize each word
df['state'] = df['state'].str.lower().str.title()
df['party_simplified'] = df['party_simplified'].str.lower().str.title()
df

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,candidatevotes,totalvotes,party_simplified
0,1976,Alabama,AL,1,63,41,"CARTER, JIMMY",DEMOCRAT,659170,1182850,Democrat
1,1976,Alabama,AL,1,63,41,"FORD, GERALD",REPUBLICAN,504070,1182850,Republican
2,1976,Alabama,AL,1,63,41,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,9198,1182850,Other
3,1976,Alabama,AL,1,63,41,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,6669,1182850,Other
4,1976,Alabama,AL,1,63,41,"HALL, GUS",COMMUNIST PARTY USE,1954,1182850,Other
...,...,...,...,...,...,...,...,...,...,...,...
4282,2020,Wyoming,WY,56,83,68,"JORGENSEN, JO",LIBERTARIAN,5768,278503,Libertarian
4283,2020,Wyoming,WY,56,83,68,"PIERCE, BROCK",INDEPENDENT,2208,278503,Other
4284,2020,Wyoming,WY,56,83,68,NaN,NaN,1739,278503,Other
4285,2020,Wyoming,WY,56,83,68,OVERVOTES,NaN,279,278503,Other


In [6]:
grouped = df.groupby(['year', 'state', 'party_simplified']).agg({
    'candidatevotes': 'sum',
    'totalvotes': 'first'
}).reset_index()

In [7]:
def winner_by_state(grouped_df): 
    return grouped_df.loc[grouped_df.groupby(['year', 'state'])['candidatevotes'].idxmax()].reset_index(drop=True)

winners = winner_by_state(grouped)

Creating the sliders that will allow the dashboard to be interactive.

In [8]:
year_slider = pn.widgets.IntSlider(name='Year slider', start=1976, end=2020, step=4, value=1976)
state_dropdown = pn.widgets.Select(name='State', options=winners['state'].unique().tolist())

In [9]:
party_colors = {
    'Democrat': 'blue',
    'Libertarian': 'yellow',
    'Other': 'gray',
    'Republican': 'red'
}

Dashboard Visualuzation Functions

In [10]:
@pn.depends(year_slider.param.value)
def map(year):
    winners_by_year = winners[winners['year'] == year]

    merged = gdf.merge(winners_by_year, left_on='name', right_on='state', how='left')
    merged['color'] = merged['party_simplified'].map(party_colors)
    merged['color'] = merged['color'].fillna('lightgray')

    plot = merged.hvplot.polygons(
        geo=True,
        color='color', 
        line_color='black',
        hover_cols=['name', 'party_simplified'],
        title=f"US Election Winners by State - {year}",
        width=800, height=500,
        xlim=(-130, -60),
        ylim=(24, 50),
        tools=['hover']
    ).opts(xaxis=None, yaxis=None)
    return plot

In [11]:
@pn.depends(year_slider.param.value)
def islands(year):
    winner_by_island = winners[(winners['year'] == year) & 
                                (winners['state'].isin(['Hawaii', 'Alaska']))]
    
    result_str = ""
    for _, row in winner_by_island.iterrows():
        result_str += f"### In {year}, the winning party in {row['state']} was the **{row['party_simplified']}** party.\n"
    
    result = pn.pane.Markdown(result_str, sizing_mode='stretch_both')
    return result

In [12]:
@pn.depends(state_dropdown)
def winning_party(state):
    state_data = winners[winners['state'] == state]

    plot = state_data.hvplot.bar(
        x='year', y='candidatevotes', 
        color='party_simplified', 
        by='party_simplified', 
        cmap=party_colors,
        title=f"Winning Party by Year in {state} (1976-2020)",
        xlabel='Year', ylabel='Winning Candidate Votes', 
        width=600, height=350, 
        legend='right').opts(yaxis=None)
    
    return plot
# remove only the labels for the bars not the xaxis label

In [13]:
@pn.depends(state_dropdown)
def candidate_percentages(state):
    state_data = grouped[grouped['state'] == state]
    state_data['percentage'] = (state_data['candidatevotes'] / state_data['totalvotes']) * 100
    
    plot = state_data.hvplot.line(
        x='year', 
        y='percentage', 
        by='party_simplified', 
        color=[party_colors[party] for party in state_data['party_simplified'].unique()],
        title=f"Percentage of Votes by Party in {state} (1976-2020)",
        xlabel='Year', 
        ylabel='Percentage', 
        width=600, height=350
    )
    return plot

Creating and Displaying the Dashboard

In [14]:
col1 = pn.Column(winning_party, candidate_percentages)  
col2 = pn.Column(map, islands) 

dashboard = pn.Column(
    pn.pane.Markdown("## US Elections Breakdown"),
    pn.Row(year_slider, state_dropdown),
    pn.Row(col1, col2)                    
)

dashboard.show()

C:\Users\andre\AppData\Local\Temp\ipykernel_6360\3955038155.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['percentage'] = (state_data['candidatevotes'] / state_data['totalvotes']) * 100


Launching server at http://localhost:59540


C:\Users\andre\AppData\Local\Temp\ipykernel_6360\3955038155.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['percentage'] = (state_data['candidatevotes'] / state_data['totalvotes']) * 100
